In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re

In [ ]:
# get url from csv file

url_list = []
with open('picasso_url_list.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        url_list.append(row[1])

url_list.remove('item_url')

In [ ]:
# log in

driver = webdriver.Chrome('/Users/ouyangling/ds/chromedriver')
time.sleep(5)
loginurl = 'https://www.liveauctioneers.com/'

#open the login page
driver.get(loginurl)
time.sleep(5)

#find and click the login link
driver.find_element_by_xpath('//*[@id="content"]/div[4]/div[3]/div[2]/div[1]/div[4]/div/a').click() 
time.sleep(5)

#find the username input and input the username
element = driver.find_element_by_id("uname")
element.send_keys('myemail@gmail.com')
time.sleep(5)

#find the password input and input password
element = driver.find_element_by_id("pw")
element.send_keys('password')
time.sleep(5)

#find login button and click
driver.find_element_by_css_selector('body > div:nth-child(17) > div > div > div.modal-right___351NU > div.modal-body___1SRyX.modal-body-default___1HKRO > div > div > button').click()
time.sleep(5)

In [ ]:
# functions to get elements

def get_item_title():
    if soup.find('h1').text:
        return soup.find('h1').text
    else:
        return None

def get_auc_end_date():
    auc_end_date_regex = re.compile('Auction ended on')
    if soup.find(text=auc_end_date_regex):
        return soup.find(text=auc_end_date_regex)
    else:
        return None

def get_sold_price():
    if soup.find(class_='item-sold-status___239o3').text:
        return soup.find(class_='item-sold-status___239o3').text
    else:
        return None

def get_est_price():
    if soup.find(class_='ended-item-estimate___bn8P7').text:
        return soup.find(class_='ended-item-estimate___bn8P7').text
    else:
        return None

def get_bid_histroy():
    if [x.text for x in soup.find_all(class_='table-cell___2tyj1')[3::2]]:
        return [x.text for x in soup.find_all(class_='table-cell___2tyj1')[3::2]]
    else:
        return None

def get_item_decription():
    if soup.find(class_='description___2Bok1').find('div').contents:
        return soup.find(class_='description___2Bok1').find('div').contents
    else:
        return None

def get_auctioneer():
    if soup.find(class_='link name___pJ4S8').text:
        return soup.find(class_='link name___pJ4S8').text
    else:
        return None

def get_auctioneer_address():
    if [x.text for x in soup.find(class_='address___fI92y').findChildren()]:
        return [x.text for x in soup.find(class_='address___fI92y').findChildren()]
    else:
        return None

In [ ]:
# scrape items

picasso_items_list = []

for i, url in enumerate(url_list):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "lxml")
    
    item_url = url
    item_title = get_item_title()
    auc_end_date = get_auc_end_date()
    sold_price = get_sold_price()
    est_price = get_est_price()
    bid_history = get_bid_histroy()
    item_description = get_item_decription()
    auctioneer = get_auctioneer()
    auctioneer_address = get_auctioneer_address()
    
    headers = ['item_url', 'item_title', 'auc_end_date', 'sold_price', 'est_price', 'bid_history', 'item_description',
                   'auctioneer', 'auctioneer_address']
    item_dict = dict(zip(headers,[item_url, item_title, auc_end_date, sold_price, est_price, bid_history, item_description, auctioneer, auctioneer_address]))
    
    picasso_items_list.append(item_dict)
    
    if i % 10 == 0:
        print(len(picasso_items_list),item_title,item_url)
    if i % 20 == 0:
        print(item_dict)
    time.sleep(5)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(picasso_items_list)
df.to_csv('picasso_data.csv', mode='a', index=False, header=False)


# header = ['auc_end_date', 'auctioneer', 'auctioneer_address', 'bid_history',
#        'est_price', 'item_description', 'item_title', 'item_url',
#        'sold_price']
